# Normalizing the Boston Dataset

In [ ]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/data/bikeshare/data/boston/boston.csv")
stations = pd.read_csv("/data/bikeshare/data/boston/stations.csv")

## Overview of the Biking Dataset

In [4]:
df.head()

,seq_id,hubway_id,status,duration,start_date,strt_statn,end_date,end_statn,bike_nr,subsc_type,zip_code,birth_date,gender
0,1,8,Closed,9,7/28/2011 10:12:00,23.0,7/28/2011 10:12:00,23.0,B00468,Registered,'97217,1976.0,Male
1,2,9,Closed,220,7/28/2011 10:21:00,23.0,7/28/2011 10:25:00,23.0,B00554,Registered,'02215,1966.0,Male
2,3,10,Closed,56,7/28/2011 10:33:00,23.0,7/28/2011 10:34:00,23.0,B00456,Registered,'02108,1943.0,Male
3,4,11,Closed,64,7/28/2011 10:35:00,23.0,7/28/2011 10:36:00,23.0,B00554,Registered,'02116,1981.0,Female
4,5,12,Closed,12,7/28/2011 10:37:00,23.0,7/28/2011 10:37:00,23.0,B00554,Registered,'97214,1983.0,Female


In [13]:
df.describe()

,seq_id,hubway_id,duration,strt_statn,end_statn,birth_date
count,1.579025e+06,1.579025e+06,1.579025e+06,1.579011e+06,1.578980e+06,350644.000000
mean,7.895130e+05,8.865317e+05,1.200280e+03,5.438039e+01,5.425603e+01,1976.295322
std,4.558254e+05,5.064783e+05,2.653539e+04,3.364295e+01,3.347219e+01,10.999742
min,1.000000e+00,8.000000e+00,-6.900000e+03,3.000000e+00,3.000000e+00,1932.000000
25%,3.947570e+05,4.465250e+05,4.120000e+02,2.700000e+01,2.900000e+01,1969.000000
50%,7.895130e+05,8.950440e+05,6.600000e+02,4.800000e+01,4.800000e+01,1979.000000
75%,1.184269e+06,1.328083e+06,1.082000e+03,7.400000e+01,7.400000e+01,1985.000000
max,1.579025e+06,1.748022e+06,1.199446e+07,1.450000e+02,1.450000e+02,1995.000000


In [14]:
df.count()

seq_id        1579025
hubway_id     1579025
status        1579025
duration      1579025
start_date    1579025
strt_statn    1579011
end_date      1579025
end_statn     1578980
bike_nr       1578559
subsc_type    1579025
zip_code      1106259
birth_date     350644
gender        1106414
dtype: int64

In [16]:
df.shape

(1579025, 13)

In [17]:
df.size

20527325

## Overview of the Station Data

In [18]:
stations.head()

,id,terminal,station,municipal,lat,lng,status
0,3,B32006,Colleges of the Fenway,Boston,42.340021,-71.100812,Existing
1,4,C32000,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616,Existing
2,5,B32012,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179,Existing
3,6,D32000,Cambridge St. at Joy St.,Boston,42.361285,-71.065140,Existing
4,7,A32000,Fan Pier,Boston,42.353412,-71.044624,Existing


In [22]:
# Unique Municipals
stations.municipal.value_counts()

Boston        97
Cambridge     28
Somerville    12
Brookline      5
Name: municipal, dtype: int64

In [25]:
# Unique Stations
stations.station.count()

142

In [27]:
print("shape: {} size: {}".format(stations.shape, stations.size))

shape: (142, 7) size: 994


__Summary__ <br>
Looking at the data we can see that the dataset consists of `13` columns and `1,579,025` rows. The data does consist of missing data so I plan to use `np.nan` to deal with the missing data. I save the iPython notebook as a Python script at the end that can be run to convert all the data into a unified format regardless of where the data is.

## Tasks

Remove irrelevant columns

In [66]:
df.columns

Index(['duration', 'start_date', 'end_date', 'strt_statn', 'end_statn',
       'subsc_type', 'birth_date', 'gender'],
      dtype='object')

In [ ]:
# backup the original DataFrame (just for trail)
# df_bak = df

In [65]:
df = df[['duration','start_date','end_date','strt_statn','end_statn','subsc_type','birth_date','gender']]
df.head()

,duration,start_date,end_date,strt_statn,end_statn,subsc_type,birth_date,gender
0,9,7/28/2011 10:12:00,7/28/2011 10:12:00,23.0,23.0,Registered,1976.0,Male
1,220,7/28/2011 10:21:00,7/28/2011 10:25:00,23.0,23.0,Registered,1966.0,Male
2,56,7/28/2011 10:33:00,7/28/2011 10:34:00,23.0,23.0,Registered,1943.0,Male
3,64,7/28/2011 10:35:00,7/28/2011 10:36:00,23.0,23.0,Registered,1981.0,Female
4,12,7/28/2011 10:37:00,7/28/2011 10:37:00,23.0,23.0,Registered,1983.0,Female


Convert duration to (minutes, int64) `GIVEN`

In [67]:
df.duration.head(3)

0      9
1    220
2     56
Name: duration, dtype: int64

Convert `starttime` and `endtime` to `datetime64`

In [68]:
df.start_date.head(3)

0    7/28/2011 10:12:00
1    7/28/2011 10:21:00
2    7/28/2011 10:33:00
Name: start_date, dtype: object

In [69]:
df.end_date.head(3)

0    7/28/2011 10:12:00
1    7/28/2011 10:25:00
2    7/28/2011 10:34:00
Name: end_date, dtype: object

In [ ]:
print("Converting start time to correct format..")
pd.to_datetime(df.start_date)
print("Conversion Completed!")

In [ ]:
print("Converting end time to correct format..")
pd.to_datetime(df.end_date)
print("Conversion Completed!")

In [ ]:
df.start_date.head(3)

Add start longitude and latitude

In [85]:
stations = stations.set_index('id')

In [137]:
df.merge(stations, left_on="strt_statn", right_index=True, how="left")

,duration,start_date,end_date,strt_statn,end_statn,subsc_type,birth_date,gender,terminal,station,municipal,lat,lng,status
0,9,7/28/2011 10:12:00,7/28/2011 10:12:00,23,23.0,Registered,1976.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
1,220,7/28/2011 10:21:00,7/28/2011 10:25:00,23,23.0,Registered,1966.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
2,56,7/28/2011 10:33:00,7/28/2011 10:34:00,23,23.0,Registered,1943.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
3,64,7/28/2011 10:35:00,7/28/2011 10:36:00,23,23.0,Registered,1981.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
4,12,7/28/2011 10:37:00,7/28/2011 10:37:00,23,23.0,Registered,1983.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
5,19,7/28/2011 10:39:00,7/28/2011 10:39:00,23,23.0,Registered,1951.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
6,24,7/28/2011 10:47:00,7/28/2011 10:47:00,23,23.0,Registered,1971.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
7,7,7/28/2011 10:48:00,7/28/2011 10:48:00,23,23.0,Registered,1971.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
8,8,7/28/2011 11:01:00,7/28/2011 11:01:00,23,23.0,Registered,1983.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
9,1108,7/28/2011 11:55:00,7/28/2011 12:13:00,47,40.0,Registered,1994.0,Male,D32010,Cross St. at Hanover St.,Boston,42.362811,-71.056067,Existing


In [138]:
df.merge(stations, left_on="end_statn", right_index=True, how="left")

,duration,start_date,end_date,strt_statn,end_statn,subsc_type,birth_date,gender,terminal,station,municipal,lat,lng,status
0,9,7/28/2011 10:12:00,7/28/2011 10:12:00,23.0,23,Registered,1976.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
1,220,7/28/2011 10:21:00,7/28/2011 10:25:00,23.0,23,Registered,1966.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
2,56,7/28/2011 10:33:00,7/28/2011 10:34:00,23.0,23,Registered,1943.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
3,64,7/28/2011 10:35:00,7/28/2011 10:36:00,23.0,23,Registered,1981.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
4,12,7/28/2011 10:37:00,7/28/2011 10:37:00,23.0,23,Registered,1983.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
5,19,7/28/2011 10:39:00,7/28/2011 10:39:00,23.0,23,Registered,1951.0,Male,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
6,24,7/28/2011 10:47:00,7/28/2011 10:47:00,23.0,23,Registered,1971.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
7,7,7/28/2011 10:48:00,7/28/2011 10:48:00,23.0,23,Registered,1971.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
8,8,7/28/2011 11:01:00,7/28/2011 11:01:00,23.0,23,Registered,1983.0,Female,B32008,Mayor Thomas M. Menino - Government Center,Boston,42.359677,-71.059364,Removed
9,1108,7/28/2011 11:55:00,7/28/2011 12:13:00,47.0,40,Registered,1994.0,Male,D32006,Lewis Wharf - Atlantic Ave.,Boston,42.363871,-71.050877,Existing


In [126]:
stations.head()

,terminal,station,municipal,lat,lng,status
id,,,,,,
3,B32006,Colleges of the Fenway,Boston,42.340021,-71.100812,Existing
4,C32000,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616,Existing
5,B32012,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179,Existing
6,D32000,Cambridge St. at Joy St.,Boston,42.361285,-71.065140,Existing
7,A32000,Fan Pier,Boston,42.353412,-71.044624,Existing
